In [1]:
import pandas as pd
import numpy as np
import yaml

## Create mapping file.

In [2]:
fp = "GAINS_ecoinvent_mapping.xlsx"
df = pd.read_excel(fp, sheet_name="ActivitySector & NFR Code")
df = df.loc[(~df["ecoinvent"].isna())&(df["ecoinvent"]!="?")]

In [3]:
df["gains_aliases"] = df["Sector"] + df["Activity"]

In [4]:
d = {}
for a in df[["Sector", "gains_aliases", "ecoinvent", "Activity", "mask name", "mask product", "EMF 30 sector"]].to_dict("records"):
    
    EU_sector_name = a["Sector"] + a["Activity"]
    EU_sector_name = EU_sector_name.replace(",", "")
    EU_sector_name = EU_sector_name.replace(";", "")
    IAM_sector_name = a["EMF 30 sector"]

    if isinstance(a["EMF 30 sector"], str):
        data = {
            "gains_aliases_EU": a["gains_aliases"],
            "gains_aliases_IAM": a["EMF 30 sector"],
            "ecoinvent_aliases": {
                "fltr": a["ecoinvent"].split(";"),
            }
        }
    else:
        data = {
            "gains_aliases_EU": a["gains_aliases"],
            "ecoinvent_aliases": {
                "fltr": a["ecoinvent"].split(";"),
            }
        }

    if isinstance(a["mask name"], str):
        data["ecoinvent_aliases"].update(
            {"mask": {"name": a["mask name"].split(";")}}
        )

    if isinstance(a["mask product"], str):
        data.update(
            {"mask": {"reference product": a["mask product"].split(";")}}
        )
    
    data["ecoinvent_aliases"]["fltr"] = [b.strip() for b in data["ecoinvent_aliases"]["fltr"] if len(b)>1]
    data["ecoinvent_aliases"]["fltr"] = list(set(data["ecoinvent_aliases"]["fltr"]))
    
    
    if EU_sector_name not in d:
        d[EU_sector_name] = data
    else:
        d[EU_sector_name]["ecoinvent_aliases"]["fltr"].extend(data["ecoinvent_aliases"]["fltr"])


In [5]:
with open('gains_mapping.yml', 'w') as outfile:
    yaml.dump(d, outfile, default_flow_style=False)

Copy file to data directory.

In [6]:
from shutil import copy2

copy2("gains_mapping.yml", "../premise/data/GAINS_emission_factors/")


'../premise/data/GAINS_emission_factors/gains_mapping.yml'

## Update exchanges

In [7]:
from premise import NewDatabase, get_regions_definition
import brightway2 as bw
bw.projects.set_current("test")

In [8]:
ndb = NewDatabase(
    scenarios = [
        {"model":"remind", "pathway":"SSP2-Base", "year":2035},
    ],
    source_db="ecoinvent 3.7.1 cutoff",
    source_version="3.7.1",
    key='tUePmX_S5B8ieZkkM7WUU2CnO8SmShwmAeWK9x2rTFo=',
)

premise v.(1, 3, 6)
+------------------------------------------------------------------+
| Warning                                                          |
+------------------------------------------------------------------+
| Because some of the scenarios can yield LCI databases            |
| containing net negative emission technologies (NET),             |
| it is advised to account for biogenic CO2 flows when calculating |
| Global Warming potential indicators.                             |
| `premise_gwp` provides characterization factors for such flows.  |
| It also provides factors for hydrogen emissions to air.          |
|                                                                  |
| Within your bw2 project:                                         |
| from premise_gwp import add_premise_gwp                          |
| add_premise_gwp()                                                |
+------------------------------------------------------------------+
+-------------

In [9]:
from premise.emissions import Emissions
em = Emissions(
    database=ndb.database,
    iam_data=ndb.scenarios[0]["iam data"],
    model="remind",
    pathway="SSP2-Base",
    year=2050,
    version="",
    gains_scenario="CLE"
)

In [10]:
em.update_emissions_in_database()

Integrating GAINS EU emission factors.
Integrating GAINS IAM emission factors.
Logging changes made to the database.
Log of changes in emissions of pollutants saved in C:\Users\davidba\premise_develop\premise\data\logs.


## Write coverage file

In [11]:
list_res = []
for ds in em.database:
    
    got_updated = False
    for exc in ds["exchanges"]:
        if exc["type"] == "biosphere" and exc.get("comment"):
            if len(exc["comment"].split(", ")) == 6:
                d = [
                    ds["name"],
                    ds["reference product"],
                    ds["unit"],
                    ds["location"],
                    exc["name"],
                ]
                d.extend(exc["comment"].split(", "))
                list_res.append(d)
                got_updated = True
    if not got_updated:
        if not any(x in ds["name"] for x in ["market for", "market group for"]):
            d = [
                ds["name"],
                ds["reference product"],
                ds["unit"],
                ds["location"],
            ]
            if ds["name"] in em.rev_gains_map_EU.keys():
                d.extend(4*[''])
                d.append(em.rev_gains_map_EU[ds["name"]])
                d.extend(2*[''])
            elif ds["name"] in em.rev_gains_map_IAM.keys():
                d.extend(4*[''])
                d.append(em.rev_gains_map_IAM[ds["name"]])
                d.extend(2*[''])
            else:
                d.extend(7*[''])
            list_res.append(d)

modified = pd.DataFrame(
    list_res,
    columns=[
        "dataset name",
        "dataset product",
        "dataset unit",
        "dataset location",
        "pollutant",
        "reduction factor",
        "old value",
        "new value",
        "GAINS variable",
        "GAINS pollutant",
        "GAINS scenario"
    ],
)

modified.to_excel("GAINS_coverage.xlsx")